In [ ]:
!pip install pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 2

In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("/content/dataset.csv")

# Parse timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], dayfirst=True)
df.sort_values(by=['ID', 'Timestamp'], inplace=True)

df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [ ]:
# Encode 'VehicleType' into weights
vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weights)

# Encode 'TrafficConditionNearby' into numerical levels
traffic_levels = {'low': 0.5, 'medium': 1.0, 'high': 1.5}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_levels)

# Convert IsSpecialDay from int (0/1) to float
df['IsSpecialDay'] = df['IsSpecialDay'].astype(float)


In [ ]:
# Constants to control the influence of each feature (you can tune these)
alpha = 1.0    # occupancy
beta = 0.5     # queue length
gamma = 0.8    # traffic
delta = 1.2    # special day
epsilon = 0.6  # vehicle weight

# Avoid divide-by-zero for Capacity
df['Capacity'] = df['Capacity'].replace(0, np.nan)

# Compute raw demand
df['RawDemand'] = (
    alpha * (df['Occupancy'] / df['Capacity']) +
    beta * df['QueueLength'] -
    gamma * df['TrafficLevel'] +
    delta * df['IsSpecialDay'] +
    epsilon * df['VehicleTypeWeight']
)

# Replace any NaN demand values with 0
df['RawDemand'] = df['RawDemand'].fillna(0)


In [ ]:
# Normalize RawDemand between 0 and 1 per lot
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-6)

df['NormalizedDemand'] = df.groupby('ID')['RawDemand'].transform(normalize)


In [ ]:
# Pricing Constants
base_price = 10
lambda_ = 1.5  # sensitivity to demand

# Final price formula
df['Price'] = base_price * (1 + lambda_ * df['NormalizedDemand'])

# Clip prices between 0.5x and 2x base price
df['Price'] = df['Price'].clip(lower=base_price * 0.5, upper=base_price * 2.0)


In [ ]:
df[['ID', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleType', 'NormalizedDemand', 'Price']].head(10)


,ID,Timestamp,Occupancy,QueueLength,TrafficLevel,IsSpecialDay,VehicleType,NormalizedDemand,Price
0,0,2016-10-04 07:59:00,61,1,0.5,0.0,car,0.0,10.0
1,1,2016-10-04 08:25:00,64,1,0.5,0.0,car,0.0,10.0
2,2,2016-10-04 08:59:00,80,2,0.5,0.0,car,0.0,10.0
3,3,2016-10-04 09:32:00,107,2,0.5,0.0,car,0.0,10.0
4,4,2016-10-04 09:59:00,150,2,0.5,0.0,bike,0.0,10.0
5,5,2016-10-04 10:26:00,177,3,0.5,0.0,car,0.0,10.0
6,6,2016-10-04 10:59:00,219,6,1.5,0.0,truck,0.0,10.0
7,7,2016-10-04 11:25:00,247,5,NaN,0.0,car,0.0,10.0
8,8,2016-10-04 11:59:00,259,5,NaN,0.0,cycle,0.0,10.0
9,9,2016-10-04 12:29:00,266,8,1.5,0.0,bike,0.0,10.0


In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category10

output_notebook()

# Select sample 3 lots for visualization
lot_ids = df['ID'].unique()[:3]

p = figure(title="Model 2: Demand-Based Dynamic Pricing",
           x_axis_label='Time',
           y_axis_label='Price ($)',
           x_axis_type='datetime',
           width=800, height=400)

colors = Category10[10]

for i, lot in enumerate(lot_ids):
    lot_df = df[df['ID'] == lot]
    source = ColumnDataSource(data={
        'x': lot_df['Timestamp'],
        'y': lot_df['Price']
    })
    p.line('x', 'y', source=source, line_width=2, color=colors[i], legend_label=f'Lot: {lot}')
    p.scatter('x', 'y', source=source, fill_color=colors[i], size=5)

p.legend.location = "top_left"
p.legend.click_policy = "hide"
show(p)


pathway integration

In [ ]:
import pathway as pw
import datetime
import pandas as pd

# Load the dataset using pandas first to create the timestamp
df_pandas = pd.read_csv("dataset.csv")
df_pandas['TimestampStr'] = df_pandas['LastUpdatedDate'] + ' ' + df_pandas['LastUpdatedTime']

# Save the modified dataframe to a new CSV file
df_pandas.to_csv("dataset_with_timestamp.csv", index=False)

# Define schema
class ParkingDataSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    TimestampStr: str # Keep TimestampStr to combine date and time

# Read the dataset as a stream
table = pw.io.csv.read(
    "dataset_with_timestamp.csv",
    schema=ParkingDataSchema,
    mode="streaming",  # required for real-time simulation
    autocommit_duration_ms=1000  # simulate a new row every 1 second
)

In [ ]:
@pw.udf
def combine_timestamp(date: str, time: str) -> str:
    return date + " " + time

table = table.select(
    table.ID,
    table.SystemCodeNumber,
    table.Capacity,
    table.Latitude,
    table.Longitude,
    table.Occupancy,
    table.VehicleType,
    table.TrafficConditionNearby,
    table.QueueLength,
    table.IsSpecialDay,
    table.LastUpdatedDate,
    table.LastUpdatedTime,
    TimestampStr=combine_timestamp(table.LastUpdatedDate, table.LastUpdatedTime)
)

In [ ]:
@pw.udf
def compute_demand(occupancy, capacity, queue, traffic, special, vehicle):
    if capacity == 0:
        return 0.0

    vehicle_weights = {'car': 1.0, 'bike': 0.7, 'truck': 1.5}
    traffic_levels = {'low': 0.5, 'medium': 1.0, 'high': 1.5}

    vt_weight = vehicle_weights.get(vehicle, 1.0)
    traffic_level = traffic_levels.get(traffic, 1.0)

    alpha = 1.0
    beta = 0.5
    gamma = 0.8
    delta = 1.2
    epsilon = 0.6

    demand = (
        alpha * (occupancy / capacity) +
        beta * queue -
        gamma * traffic_level +
        delta * special +
        epsilon * vt_weight
    )
    return max(0.0, demand)


In [ ]:
@pw.udf
def price_from_demand(demand):
    base_price = 10
    lambda_ = 1.5
    price = base_price * (1 + lambda_ * demand)
    return min(max(price, base_price * 0.5), base_price * 2.0)


In [ ]:
result_table = table.select(
    ID=table.ID,
    Timestamp=table.TimestampStr,
    Demand=compute_demand(
        table.Occupancy,
        table.Capacity,
        table.QueueLength,
        table.TrafficConditionNearby,
        table.IsSpecialDay,
        table.VehicleType
    )
)

result_table += result_table.select(
    Price=price_from_demand(result_table.Demand)
)


In [ ]:
pw.io.jsonlines.write(result_table, "output_pathway.jsonl")
pw.run()


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           506                                     
    FileSystem…            0             0        18368    output          506          0                          
                                                                                                                   
                                                            Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    